In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-05 01:47:41.196694: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 01:47:41.402823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 01:47:42.258413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_union_all_S"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [21]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "slate").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "slate")
train_target = filtered_target
test_target = filtered_test_target
train_target[2]
 

{'promptID': 139803,
 'pairID': '139803n',
 'premise': "The word that is used around Doug is 'operator.",
 'premise_binary_parse': '( ( ( The word ) ( that ( is ( used ( around Doug ) ) ) ) ) ( ( ( is ` ) operator ) . ) )',
 'premise_parse': '(ROOT (S (NP (NP (DT The) (NN word)) (SBAR (WHNP (WDT that)) (S (VP (VBZ is) (VP (VBN used) (PP (IN around) (NP (NNP Doug)))))))) (VP (VBZ is) (`` `) (NP (NN operator))) (. .)))',
 'hypothesis': "People only choose to say the word 'operator' when talking to Doug",
 'hypothesis_binary_parse': "( People ( only ( choose ( to ( ( say ( the ( word ( ` ( operator ' ) ) ) ) ) ( when ( talking ( to Doug ) ) ) ) ) ) ) )",
 'hypothesis_parse': "(ROOT (S (NP (NNS People)) (VP (ADVP (RB only)) (VBP choose) (S (VP (TO to) (VP (VB say) (NP (DT the) (NN word) (`` `) (NN operator) ('' ')) (SBAR (WHADVP (WRB when)) (S (VP (VBG talking) (PP (TO to) (NP (NNP Doug))))))))))))",
 'genre': 'slate',
 'label': 1}

In [22]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [23]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig

union_config = ConfigUnion(
    PrefixTuningConfig(prefix_length=10),
    LoRAConfig(r=8, alpha=16),

    SeqBnInvConfig(reduction_factor=2),
)
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_union_all_S          union            17,539,712      16.107       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 18131840 || all params: 127054010 || trainable%: 14.2709702747674


In [24]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77306
})

In [25]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [26]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 23975
})

In [27]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] notieri in america goes to a big public high school, but everyone goes to a high school governed by [MASK] hierarchy of popularity and cliques. [SEP] everyone has big public schools [MASK]. [SEP] [CLS] i am [MASK] gay. [SEP] i am gay. [SEP] [CLS] the word that is used around [MASK] is'[MASK]. [SEP] [MASK] harmon choose to say the [MASK]'operator'when talking to doug [SEP] [CLS] however, [MASK] [MASK] - bound colorado students take a [MASK] college entrance exam, making the [MASK] an unreliable measure of school quality. [SEP] some compute take [MASK] college entrance exams. [SEP] [CLS] their issuing of [MASK] - term buy or [MASK] - term hold'

'>>> recommendations obviously intensifies pressure on companies to meet and beat earnings expectations at [MASK] costs. [SEP] most companies will meet their earning expectations. [SEP] [CLS] well, my wife nearly [MASK] brownish to see [MASK] castigate [MASK] in public, no less on the internet [MASK] [SEP] my wife nearly fainted [MASK

In [28]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 21577
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2398
    })
})

In [29]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/75 [00:00<?, ?it/s]

>>> Perplexity: 16.82


In [31]:
trainer.train()

  0%|          | 0/13500 [00:00<?, ?it/s]

{'loss': 2.372, 'grad_norm': 1.9281458854675293, 'learning_rate': 9.866153846153846e-05, 'epoch': 1.0}


  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 2.021080255508423, 'eval_runtime': 4.6185, 'eval_samples_per_second': 519.212, 'eval_steps_per_second': 16.239, 'epoch': 1.0}
{'loss': 2.0975, 'grad_norm': 1.806840181350708, 'learning_rate': 9.347692307692308e-05, 'epoch': 2.0}


  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 1.7445213794708252, 'eval_runtime': 4.6355, 'eval_samples_per_second': 517.313, 'eval_steps_per_second': 16.18, 'epoch': 19.0}


Overwriting existing adapter 'mlm_union_all_S'.


{'train_runtime': 1752.3706, 'train_samples_per_second': 246.261, 'train_steps_per_second': 7.704, 'train_loss': 1.911978156255235, 'epoch': 19.0}


TrainOutput(global_step=12825, training_loss=1.911978156255235, metrics={'train_runtime': 1752.3706, 'train_samples_per_second': 246.261, 'train_steps_per_second': 7.704, 'train_loss': 1.911978156255235, 'epoch': 19.0})

In [32]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/75 [00:00<?, ?it/s]

>>> Perplexity: 5.68


In [33]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for Slate domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 16.82

The results after:
>>> Perplexity: 5.68